In [ ]:
#make all imports 
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os

In [ ]:
#parse command line arguments 
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
    help="path to input image") #path to input image containing faces 
ap.add_argument("-f", "--face", type=str,
    default="face_detector",
    help="path to face detector model directory") #path to face detector model directory
ap.add_argument("-m", "--model", type=str,
    default="mask_detector.model",
    help="path to trained face mask detector model") 
ap.add_argument("-c", "--confidence", type=float, default=0.5,
    help="minimum probability to filter weak detections") 
args = vars(ap.parse_args())

In [ ]:
# load serialized face detector model from disk 
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join([args["face"], "deploy.prototxt"])
weightsPath = os.path.sep.join([args["face"],
    "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
model = load_model(args["model"])

In [ ]:
#load and preprocess an input image 

# load the input image from disk, clone it, and grab the image spatial dimensions
image = cv2.imread(args["image"])
orig = image.copy()
(h, w) = image.shape[:2]

#pass image through the network and obtain face detections
blurb = cv2.dnn.blobFromImage(image, 1.0, (300, 300),(104.0, 177.0, 123.0)) #pre-processing done by OpenCV function
print("[INFO] computing face detections...") #finds where the face is located in the image 
net.setInput(blurb)
detections = net.forward()

#Once we know where the face is located in each image, ensure they meet the --confidence threshold before extracting the faceROI's 
for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    
    if confidence > args["confidence"]:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h]) #commpute coordinated of the objec's bounding box 
        (startX, startY, endX, endY) = box.astype("int")
        
        (startX, startY) = (max(0, startX), max(0, startY)) #ensure bounding box is within the frame dimensions
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
    

In [ ]:
#Extract the face ROI and pre-process it
face = image[startY:endY, startX:endX] #extracted using NumPy slicing
face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
face = cv2.resize(face, (224, 224))
face = img_to_array(face)
face = preprocess_input(face)
face = np.expand_dims(face, axis=0)

#pass face through model to detect whether it's wearing a mask or not
(mask, withoutMask) = model.predict(face)[0]

In [ ]:
#Display the result 

label = "Mask" if mask > withoutMask else "No Mask"
color = (0, 255, 0) if label == "Mask" else (0, 0, 255) #either green or red
label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
cv2.putText(image, label, (startX, startY - 10), #OpenCV drawing functions
    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
cv2.imshow("Output", image)
cv2.waitKey(0)